In [1]:
import pandas as pd
from datetime import datetime

Strawberries are planted around October 1 and ready for harvest by the end of January. What is the mean risk of frost and freeze, defined as the mean number of days per month over the period 1991-2020 that the temperature has been observed to be less than or equal to 32 and 28 degrees Fahrenheit, respectively, that might damage the plants for each month during the October - January period? (25 points)

In [2]:
def get_station_data(ghcn_id, start_year, end_year):

    try:
        df = pd.read_parquet(
            f"s3://noaa-ghcn-pds/parquet/by_station/STATION={ghcn_id}/",
            storage_options={"anon": True},
        )
    except Exception as exception:
        print(f"Error loading data for station {ghcn_id}: {exception}")
        return None
    
    df['DATE'] = pd.to_datetime(df['DATE'].apply(lambda x: datetime.strptime(x, '%Y%m%d')))
    df = df.set_index('DATE').sort_index() 

    df = df[(df.index >= pd.to_datetime(f'{start_year}-01-01')) & (df.index <= pd.to_datetime(f'{end_year}-12-31'))]

    df_tobs = df.loc[df['ELEMENT'] == 'TMIN']
    df_tobs = df_tobs['DATA_VALUE']/10. # convert to Celsius

    df_tobs_f = df_tobs * 9./5. + 32. # convert to Fahrenheit

    strawberry_season = df_tobs_f[df_tobs_f.index.month.isin([10, 11, 12, 1])]

    frost = (strawberry_season <= 32.).groupby([strawberry_season.index.year, strawberry_season.index.month]).sum()
    freeze = (strawberry_season <= 28.).groupby([strawberry_season.index.year, strawberry_season.index.month]).sum()

    frost_mean = frost.groupby(level=1).mean()
    freeze_mean = freeze.groupby(level=1).mean()    

    res = pd.DataFrame({
        'frost_mean_risk': frost_mean,
        'freeze_mean_risk': freeze_mean
    })

    



    return res

In [3]:
df_risk = get_station_data('USC00087205', 1991, 2020)
print(df_risk)

      frost_mean_risk  freeze_mean_risk
DATE                                   
1            1.866667          0.500000
10           0.000000          0.000000
11           0.033333          0.000000
12           0.600000          0.166667


To begin to explore the seasonal to sub-seasonal prediction of freeze events at this site, using code you adapt from Module 4, we're going to try to relate these cold events to the El Nino Southern Oscillation (ENSO).  You have a hypothesis that ENSO is related to seasonal prediction of freeze events, but you don't know which region to choose for calculating your anomalies.  The problem is that there are many ENSO indicies that represent forcing across the eastern and central Pacific: which SST forcing region is most related to cold conditions in central Florida? 
NOAA CPC has calculated mean SSTs and anomalies in each of these 4 regions (cpc.ncep.noaa.gov/data/indices/sstoi.indicesLinks to an external site.). Using the temperature anomalies computed in the file, determine which ENSO index (NINO1+2, NINO3, NINO4, and NINO3.4) is best correlated (i.e., has the highest absolute value of Pearson's correlation coefficient) with the number of days per month < 28 degrees F.  (25 points)

In [4]:
df_sst = pd.read_csv('https://www.cpc.ncep.noaa.gov/data/indices/sstoi.indices',sep=r'\s+', engine='python')
df_sst['year'] = df_sst['YR']
df_sst['month'] = df_sst['MON']
df_sst

,YR,MON,NINO1+2,ANOM,NINO3,ANOM.1,NINO4,ANOM.2,NINO3.4,ANOM.3,year,month
0,1982,1,24.28,-0.24,25.84,0.17,28.01,-0.21,26.65,0.08,1982,1
1,1982,2,25.38,-0.72,26.26,-0.11,27.99,-0.11,26.54,-0.20,1982,2
2,1982,3,25.22,-1.38,26.92,-0.25,28.18,-0.05,27.09,-0.14,1982,3
3,1982,4,24.57,-1.16,27.52,-0.05,28.61,0.10,27.83,0.02,1982,4
4,1982,5,24.00,-0.62,27.70,0.49,29.19,0.40,28.37,0.49,1982,5
...,...,...,...,...,...,...,...,...,...,...,...,...
522,2025,7,22.29,0.46,25.92,0.04,28.84,0.05,27.24,-0.06,2025,7
523,2025,8,21.09,0.23,24.97,-0.24,28.63,-0.06,26.58,-0.33,2025,8
524,2025,9,20.40,-0.18,24.60,-0.41,28.41,-0.27,26.32,-0.44,2025,9
525,2025,10,20.83,-0.04,24.74,-0.35,28.36,-0.33,26.29,-0.48,2025,10


In [5]:
try:
        df = pd.read_parquet(
            f"s3://noaa-ghcn-pds/parquet/by_station/STATION=USC00087205/",
            storage_options={"anon": True},
        )
except Exception as exception:
        print(f"Error loading data: {exception}")

In [6]:
df['DATE'] = pd.to_datetime(df['DATE'].apply(lambda x: datetime.strptime(x, '%Y%m%d')))
df = df.set_index('DATE').sort_index()  #we need to sort by time because the files are sorted to be arbitrary

df_tmin = df.loc[df['ELEMENT'] == 'TMIN']

In [7]:
df_tmin

,ID,DATA_VALUE,M_FLAG,Q_FLAG,S_FLAG,OBS_TIME,ELEMENT
DATE,,,,,,,
1892-09-01,USC00087205,206,None,None,6,None,TMIN
1892-09-02,USC00087205,206,None,None,6,None,TMIN
1892-09-03,USC00087205,211,None,None,6,None,TMIN
1892-09-04,USC00087205,217,None,None,6,None,TMIN
1892-09-05,USC00087205,211,None,None,6,None,TMIN
...,...,...,...,...,...,...,...
2025-12-10,USC00087205,128,None,None,H,1600,TMIN
2025-12-11,USC00087205,100,None,None,H,1600,TMIN
2025-12-12,USC00087205,67,None,None,H,1600,TMIN


In [8]:
df_tmin['DATA_VALUE'] = df_tmin['DATA_VALUE']/10. # convert to Celsius
df_tmin['DATA_VALUE_F'] = df_tmin['DATA_VALUE'] * 9./5. + 32. # convert to Fahrenheit
df_tmin

/tmp/ipykernel_1394269/3313822586.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmin['DATA_VALUE'] = df_tmin['DATA_VALUE']/10. # convert to Celsius
/tmp/ipykernel_1394269/3313822586.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmin['DATA_VALUE_F'] = df_tmin['DATA_VALUE'] * 9./5. + 32. # convert to Fahrenheit


,ID,DATA_VALUE,M_FLAG,Q_FLAG,S_FLAG,OBS_TIME,ELEMENT,DATA_VALUE_F
DATE,,,,,,,,
1892-09-01,USC00087205,20.6,None,None,6,None,TMIN,69.08
1892-09-02,USC00087205,20.6,None,None,6,None,TMIN,69.08
1892-09-03,USC00087205,21.1,None,None,6,None,TMIN,69.98
1892-09-04,USC00087205,21.7,None,None,6,None,TMIN,71.06
1892-09-05,USC00087205,21.1,None,None,6,None,TMIN,69.98
...,...,...,...,...,...,...,...,...
2025-12-10,USC00087205,12.8,None,None,H,1600,TMIN,55.04
2025-12-11,USC00087205,10.0,None,None,H,1600,TMIN,50.00
2025-12-12,USC00087205,6.7,None,None,H,1600,TMIN,44.06


In [9]:
df_tmin = df_tmin[(df_tmin.index >= pd.to_datetime('1982-01-01')) & (df_tmin.index <= pd.to_datetime('2025-11-30'))]

In [10]:
df_tmin

,ID,DATA_VALUE,M_FLAG,Q_FLAG,S_FLAG,OBS_TIME,ELEMENT,DATA_VALUE_F
DATE,,,,,,,,
1982-01-01,USC00087205,14.4,None,None,0,1800,TMIN,57.92
1982-01-02,USC00087205,13.3,None,None,0,1800,TMIN,55.94
1982-01-03,USC00087205,13.9,None,None,0,1800,TMIN,57.02
1982-01-04,USC00087205,13.3,None,None,0,1800,TMIN,55.94
1982-01-05,USC00087205,6.7,None,None,0,1800,TMIN,44.06
...,...,...,...,...,...,...,...,...
2025-11-26,USC00087205,17.2,None,None,H,1600,TMIN,62.96
2025-11-27,USC00087205,17.2,None,None,H,1600,TMIN,62.96
2025-11-28,USC00087205,6.7,None,None,H,1600,TMIN,44.06


In [11]:
freeze_days = (df_tmin['DATA_VALUE_F'] < 28.).groupby([df_tmin.index.year, df_tmin.index.month]).sum()

In [12]:
freeze_df = freeze_days.to_frame(name='freeze_days')
freeze_df.index.names = ['year', 'month']

In [13]:
freeze_df

freeze_days
year month             
1982 1                1
     2                0
     3                0
     4                0
     5                0
...                 ...
2025 7                0
     8                0
     9                0
     10               0
     11               0

[524 rows x 1 columns]

In [14]:
merged = freeze_df.merge(df_sst[['year', 'month', 'ANOM', 'ANOM.1', 'ANOM.2', 'ANOM.3']], 
                         on=['year', 'month'], 
                         how='inner')

In [15]:
merged

,year,month,freeze_days,ANOM,ANOM.1,ANOM.2,ANOM.3
0,1982,1,1,-0.24,0.17,-0.21,0.08
1,1982,2,0,-0.72,-0.11,-0.11,-0.20
2,1982,3,0,-1.38,-0.25,-0.05,-0.14
3,1982,4,0,-1.16,-0.05,0.10,0.02
4,1982,5,0,-0.62,0.49,0.40,0.49
...,...,...,...,...,...,...,...
519,2025,7,0,0.46,0.04,0.05,-0.06
520,2025,8,0,0.23,-0.24,-0.06,-0.33
521,2025,9,0,-0.18,-0.41,-0.27,-0.44
522,2025,10,0,-0.04,-0.35,-0.33,-0.48


In [16]:
merged.corr(method='pearson')

,year,month,freeze_days,ANOM,ANOM.1,ANOM.2,ANOM.3
year,1.000000,-0.004870,-0.046722,0.044847,0.076358,0.204021,0.060388
month,-0.004870,1.000000,-0.073342,0.014590,0.000026,0.002844,0.001245
freeze_days,-0.046722,-0.073342,1.000000,-0.078137,-0.118407,-0.098660,-0.104823
ANOM,0.044847,0.014590,-0.078137,1.000000,0.810746,0.442799,0.639311
ANOM.1,0.076358,0.000026,-0.118407,0.810746,1.000000,0.739764,0.940460
ANOM.2,0.204021,0.002844,-0.098660,0.442799,0.739764,1.000000,0.884520
ANOM.3,0.060388,0.001245,-0.104823,0.639311,0.940460,0.884520,1.000000


In [17]:
# ANOM.1 is the strongest correlation -> Nino3